### this notebook scrapes raw course data from coursereport.com
#### (bootcamps)

In [ ]:
from selenium import webdriver
import time
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
%%capture
tqdm.pandas()

## coursereport

In [ ]:
driver = webdriver.Chrome()

urls = []
for page_n in tqdm(range(1, 18)):
    url = f"https://www.coursereport.com/tracks/data-science?page={page_n}"
    driver.get(url)
    schools = driver.find_elements_by_class_name('school-li')
    urls += [s.find_element_by_tag_name('a').get_attribute('href')
                    for s in schools]
print(f'Collected URLs for {len(urls)} schools.\nScraping . . .')

In [ ]:
driver = webdriver.Chrome()

urls = []
for page_n in tqdm(range(1, 18)):
    url = f"https://www.coursereport.com/tracks/data-science?page={page_n}"
    driver.get(url)
    schools = driver.find_elements_by_class_name('school-li')
    urls += [s.find_element_by_tag_name('a').get_attribute('href')
                    for s in schools]
print(f'Collected URLs for {len(urls)} schools.\nScraping . . .')

data = pd.DataFrame()

for school_url in tqdm(urls):

    driver.get(school_url)

    try: driver.find_element_by_id('match-close-btn').click()
    except: pass

    header = driver.find_element_by_class_name('main-header')
    school_name = header.find_element_by_tag_name('h1').text

    rating = header.find_element_by_class_name('rating-number')
    review_score = rating.find_elements_by_tag_name('span')[0].text
    n_reviews = rating.find_elements_by_tag_name('span')[2].text

    print('\n', school_name.upper(), f'({review_score} – {n_reviews} reviews)')

    school_data = []

    courses = driver.find_elements_by_class_name('course-card')

    for c in courses:
        
        try:
            driver.find_element_by_id('match-close-btn').click()
            print('**closing popup**')
        except: pass
        
        try:
            c_data = {}

            c_data['organization'] = school_name
            c_data['review_score'] = review_score
            c_data['n_reviews'] = n_reviews

            c_data['course_name'] = c.find_element_by_tag_name('h3').text

            try: c_data['link'] = c.find_element_by_class_name('btn-apply'
                                    ).find_element_by_tag_name('a').get_attribute('href')
            except: pass

            details = c.find_element_by_class_name('details')

            c_data['topics'] = ', '.join([t.text for t in details.find_elements_by_tag_name('a')])

            c_data.update({d.find_element_by_tag_name('span').get_attribute('title'): d.text
                             for d in details.find_elements_by_class_name("type")})

            try: c_data['hours_per_week'] = details.find_element_by_class_name('hours-week-number').text
            except: pass

            main = c.find_element_by_class_name('content')
            rows = main.find_elements_by_tag_name('tr')
            c_data.update({r.find_element_by_tag_name('th').text: r.find_element_by_tag_name('td').text for r in rows})

            print('  >', c_data['course_name'])

            school_data.append(c_data)
        except: pass
        
    data = pd.concat([data, pd.DataFrame(school_data)], sort=False)
    data = data[(data.course_name.str.lower().str.contains('|'.join(focus_topics)))|
                (data.topics.str.lower().str.contains('|'.join(focus_topics)))]
    data.to_csv('scraped_coursereport.csv', index=False)
    driver.quit()
    print('Done... saved.')